In [1]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import random
import string
import os

# 配置参数
WIDTH = 100                 # 图片宽度
HEIGHT = 35                 # 图片高度
FONT_PATHS = [              # 尝试加载的字体路径
    'arial.ttf',
    '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf',
    '/Library/Fonts/Arial.ttf',
    'C:/Windows/Fonts/arial.ttf'
]
OUTPUT_FILE = 'captcha.jpg'  # 输出文件名

def generate_captcha():
    # 创建图像和绘图对象
    image = Image.new('RGB', (WIDTH, HEIGHT), (255, 255, 255))
    draw = ImageDraw.Draw(image)

    # 查找可用字体
    font_path = None
    for path in FONT_PATHS:
        if os.path.exists(path):
            font_path = path
            break
    if not font_path:
        raise FileNotFoundError("未找到可用字体文件")

    # 生成随机验证码（4位）
    characters = ''.join(random.choices(string.digits + string.ascii_letters, k=4))

    # 第一次添加干扰元素（底层）
    _add_noise(draw, density=30)
    _add_interference_lines(draw, num_lines=3)

    # 绘制验证码字符
    for i, char in enumerate(characters):
        # 随机字符参数
        font_size = random.randint(20, 30)
        rotation = random.randint(-25, 25)
        x_offset = random.randint(-8, 8)
        y_offset = random.randint(-8, 8)
        char_color = (random.randint(0, 150), random.randint(0, 150), random.randint(0, 150))

        # 创建字符图像
        char_image = _create_char_image(char, font_path, font_size, char_color, rotation)
        
        # 计算位置（四等分区域）
        x = i * (WIDTH/4) + (WIDTH/4 - char_image.width)/2 + x_offset
        y = (HEIGHT - char_image.height)/2 + y_offset
        
        # 合并到主图像
        image.paste(char_image, (int(x), int(y)), char_image)

    # 第二次添加干扰元素（上层）
    _add_noise(draw, density=50)
    _add_interference_lines(draw, num_lines=2)
    image = image.filter(ImageFilter.SMOOTH)

    # 保存结果
    image.save(OUTPUT_FILE)
    print(f"验证码已生成到 {OUTPUT_FILE}，内容：{characters}")

def _create_char_image(char, font_path, font_size, color, rotation):
    """生成单个字符的旋转图像"""
    font = ImageFont.truetype(font_path, font_size)
    
    # 创建临时图像（足够大以容纳旋转）
    temp_image = Image.new('RGBA', (font_size*2, font_size*2), (0, 0, 0, 0))
    temp_draw = ImageDraw.Draw(temp_image)
    
    # 绘制字符（居中）
    bbox = temp_draw.textbbox((0,0), char, font=font)
    x = (temp_image.width - (bbox[2]-bbox[0]))/2 - bbox[0]
    y = (temp_image.height - (bbox[3]-bbox[1]))/2 - bbox[1]
    temp_draw.text((x, y), char, font=font, fill=color)
    
    # 旋转并返回
    return temp_image.rotate(rotation, expand=True, resample=Image.BILINEAR, fillcolor=(0,0,0,0))

def _add_noise(draw, density=100):
    """添加噪点"""
    for _ in range(density):
        x = random.randint(0, WIDTH)
        y = random.randint(0, HEIGHT)
        draw.point((x, y), fill=(
            random.randint(150, 255),
            random.randint(150, 255),
            random.randint(150, 255)
        ))

def _add_interference_lines(draw, num_lines=3):
    """添加干扰线"""
    for _ in range(num_lines):
        start = random.randint(0, WIDTH), random.randint(0, HEIGHT)
        end = random.randint(0, WIDTH), random.randint(0, HEIGHT)
        draw.line([start, end], 
                 fill=(random.randint(100, 200), 
                      random.randint(100, 200), 
                      random.randint(100, 200)), 
                 width=random.randint(1, 2))

if __name__ == '__main__':
    generate_captcha()

验证码已生成到 captcha.jpg，内容：WG22
